In [ ]:
!pip install -q -U google-generativeai chromadb pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.0/165.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.4/725.4 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 18.1 MB/s et

In [ ]:
import google.generativeai as genai
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
import pdfplumber as plmb

In [ ]:
from google.colab import userdata
key=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=key)
safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]
model = genai.GenerativeModel('models/gemini-1.5-flash', safety_settings=safety_settings)

from google.colab import files
def upload():
  uploaded = files.upload()
  if uploaded:
    for file_name in uploaded.keys():
      print(f"Uploaded: {file_name}")
      return file_name
  else:
    return None


def extraction(pdf_path):
  with plmb.open(pdf_path) as pdf:
    text=""
    for page in pdf.pages:
      text+=page.extract_text()
  return text


def embed(text):
  embed_data=genai.embed_content(model='models/text-embedding-004',content=[text],output_dimensionality=384)
  return embed_data['embedding'][0]

def chromaup(data,name):
  client=chromadb.Client()
  embeddings = embed(data)
  collection=client.get_or_create_collection(name)
  collection.add(
              ids=["segment"],
                documents=[data],
                embeddings=[embeddings]
            )
  return collection

def embed_q(query):
  embed_data=genai.embed_content(model='models/text-embedding-004',content=[query],output_dimensionality=384)
  return embed_data['embedding'][0]


def search_doc(query_text,collection_name):
  query_embed=embed_q(query_text)
  client=chromadb.Client()
  coll=client.get_or_create_collection(collection_name)
  results=coll.query(
      query_embeddings=[query_embed],
      n_results=1
  )
  return results

def generation(query,doc):
  Prompt=f"Document:{doc}\n\nQuestion:{query}"
  response=model.generate_content(Prompt).text
  return response


In [ ]:
file = upload()
text=extraction(file)
text_db=chromaup(text,'text_file1')

Saving Pratham_Sinha_Resume (1).pdf to Pratham_Sinha_Resume (1).pdf
Uploaded: Pratham_Sinha_Resume (1).pdf


In [ ]:
que=input()
resp=search_doc(que,'text_file1')
ans=generation(que,resp['documents'][0][0])
print(ans)

intern?
Pratham Sinha has had one internship:

* **Krutrim SI Designs Private Limited** (December 2023 - January 2024) as an **Artificial Intelligence and Prompt Intern**. 

